In [59]:
from cdae.models import cdae_my
import tensorflow as tf
import pandas as pd
import numpy as np
import os,sys
from scipy.sparse import *
from rec_eval_model import precision_recall_at_ks

In [60]:
def load_data(csv_file, shape):
    tp = pd.read_csv(csv_file)
    rows, cols = np.array(tp['uid']), np.array(tp['sid'])
    seq = np.concatenate((rows[:, None], cols[:, None], np.ones((rows.size, 1), dtype='int') ), axis=1)
    data = csr_matrix((np.ones_like(rows), (rows, cols)), dtype=np.float32, shape=shape)
    return data, seq



In [61]:
DATA_DIR = 'data/ml-100k/pro'
uid_fname = 'unique_uid.txt'
sid_fname = 'unique_sid.txt'
rating_fname = 'train.csv'
test_fname ='test.csv'
unique_uid = list()
with open(os.path.join(DATA_DIR, uid_fname), 'r') as f:
    for line in f:
        unique_uid.append(line.strip())
unique_sid = list()

with open(os.path.join(DATA_DIR, sid_fname), 'r') as f:
    for line in f:
        unique_sid.append(line.strip())
n_items = len(unique_sid)
n_users = len(unique_uid)

train_data, train_raw = load_data(os.path.join(DATA_DIR, rating_fname),(n_users,n_items))
test_data, test_raw = load_data(os.path.join(DATA_DIR, test_fname),(n_users,n_items))


In [62]:
train_data,test_data = train_data.todense(),test_data.todense()

In [65]:
model = cdae_my.mymodel(n_users, n_items, latent_dim = 100, learning_rate = 0.15,
                        keep_prob = 0.4, activation = 'sigmoid', use_user_bias = False)

In [66]:
for i in range(50):
    model.run_epoch(16,train_data)

In [69]:
ks = [1,3,5,10,15,30,50]
ret = precision_recall_at_ks(model,train_data,test_data,ks = ks)
for k,pr,rc in zip(*[ks , ret[0], ret[1]]):
    print("[K : %d] Precision@%d : %0.4f\t Recall@%d : %0.4f" % (k, k, pr, k, rc))
    
    

[K : 1] Precision@1 : 0.3535	 Recall@1 : 0.0479
[K : 3] Precision@3 : 0.2845	 Recall@3 : 0.1034
[K : 5] Precision@5 : 0.2490	 Recall@5 : 0.1454
[K : 10] Precision@10 : 0.2023	 Recall@10 : 0.2251
[K : 15] Precision@15 : 0.1763	 Recall@15 : 0.2885
[K : 30] Precision@30 : 0.1333	 Recall@30 : 0.4189
[K : 50] Precision@50 : 0.1061	 Recall@50 : 0.5295


/home/ita/code/recsys/factor-based-recommenderSystems/rec_eval_model.py:19: RuntimeWarning: invalid value encountered in true_divide
  recalls.append(np.nanmean(tmp/test_data.sum(axis=1)))
